<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 2 : Modèle de Drude</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

- Michael Hudon
- Émile Poirier
- Justin Binette
- Sébastien Rouleau

# Introduction # 
Cette deuxième partie du travail implémente une simulation 2D du modèle de Drude pour décrire le mouvement des électrons libres et indépendants dans les métaux. Des sphères dures sont encore utilisées pour représenter les particules, mais maintenant de deux types différents afin de différencier les coeurs ioniques immobiles des électrons sur leurs trajectoires balistiques. Les collisions entre les deux doivent donc être inélastiques. Les questions sur cette simulation, d'abord de cinétique puis de dynamique des électrons en présence d'une force externe appliquée au cristal, vérifieront quelques résultats analytiques du modèle de Drude obtenus en classe et/ou dans le manuel de cours Ashcroft/Mermin.

- **La remise du présent _Jupyter Notebook_ ET celui de la 1<sup>re</sup> partie, ainsi que tout autre fichier de code produit, se fait sur Gradescope en n'oubliant pas d'y indiquer tous les membres de votre équipe.**

# 2<sup>e</sup> partie - Modèle de Drude #

Faites une copie du script `tds2Danimation_hXX.py` et modifiez-le pour obtenir une simulation cinétique d'un gaz d'électrons libres dans la matière cristalline selon le modèle de Drude. Spécifiquement selon les pp.4-6 du manuel Ashcroft/Mermin,
1. ajoutez un deuxième type de sphères fixes réparties périodiquement dans la boîte d'animation: celles-ci représenteront les coeurs ioniques,
2. éliminez les collisions entre les sphères mobiles qui représentent maintenant les électrons de conduction indépendants et libres,
3. en faisant appel à la température du gaz, ajoutez des collisions inélastiques entre les électrons libres et les coeurs ioniques fixes. La quantité de mouvement $\vec{p}$ n'est alors PAS conservée et il faut appliquer les hypothèses spécifiques du modèle de Drude à la sortie de chaque collision, notamment: 
- la direction de $\vec{p}$ doit être aléatoire,
- la norme $||\vec{p}||$ est déterminée par la distribution de Maxwell-Boltzmann.

### Votre simulation ###

VII. _(25 points)_ &mdash; Écrivez un appel de votre nouveau script pour l'exécuter avec la cellule suivante:

In [3]:
!python Druide_(Question7-8).py

Figure(1000x500)


### Question statistique ###

VIII. _(5 points)_ &mdash; Vérifiez numériquement et graphiquement que l'amortissement de la quantité de mouvement moyenne des électrons suit l'équation exponentielle dérivée analytiquement en classe, soit $\langle p(t)\rangle =\langle p(t_0)\rangle \,e^{-t/\tau}$, et comparez-y le comportement d'un seul électron.

In [ ]:
from vpython import *
import numpy as np
from scipy.stats import maxwell
import matplotlib.pyplot as plt  # Pour le tracé du graphique

# Déclaration de variables influençant le temps d'exécution de la simulation
Nions = 25  # Nombre de cœurs ioniques
Rion = 0.02  # Taille fictive des ions
Natoms = 200  # Nombre d'atomes
dt = 1E-8  # Pas d'incrémentation temporelle

# Déclaration de variables physiques
DIM = 2  # Nombre de degrés de liberté de la simulation
mass = 9.109e-31  # Masse d'un atome
Ratom = 0.01  # Taille des atomes
k = 1.4E-23  # Constante de Boltzmann
T = 1000  # Température ambiante augmentée à 1000K

#### CANEVAS DE FOND ####
L = 1  # Conteneur en forme de cube
gray = color.gray(0.7)  # Couleur des bords du conteneur
animation = canvas(width=750, height=500)
animation.range = L

#### ARÊTES DE BOÎTE 2D ####
d = L / 2 + Ratom
r = 0.005
cadre = curve(color=gray, radius=r)
cadre.append([vector(-d, -d, 0), vector(d, -d, 0), vector(d, d, 0), vector(-d, d, 0), vector(-d, -d, 0)])

#### POSITION ET QUANTITÉ DE MOUVEMENT INITIALE DES SPHÈRES ####
Atoms = []
p = []
apos = []
pavg = sqrt(2 * mass * (DIM / 2) * k * T)

for i in range(Natoms):
    x = L * random() - L / 2
    y = L * random() - L / 2
    z = 0
    Atoms.append(simple_sphere(pos=vector(x, y, z), radius=Ratom, color=gray))
    apos.append(vec(x, y, z))
    phi = 2 * pi * random()
    px = pavg * cos(phi)
    py = pavg * sin(phi)
    p.append(vector(px, py, 0))

# Initialisation des cœurs ioniques
Ions = []
ion_pos = []

grid_size = int(sqrt(Nions))
dist = L / grid_size

for i in range(grid_size):
    for j in range(grid_size):
        if len(Ions) >= Nions:
            break
        x = -L / 2 + i * dist + dist / 2
        y = -L / 2 + j * dist + dist / 2
        ion = simple_sphere(pos=vector(x, y, 0), radius=Rion, color=color.red)
        Ions.append(ion)
        ion_pos.append(vector(x, y, 0))

# Fonction pour identifier les collisions entre électrons et ions
def checkIonCollisions():
    hitlist = []
    r2 = (Ratom + Rion) ** 2
    for i in range(Natoms):
        ai = apos[i]
        for j in range(len(Ions)):
            aj = ion_pos[j]
            dr = ai - aj
            if mag2(dr) < r2:
                hitlist.append([i, j])
    return hitlist

# Fonction pour générer une quantité de mouvement aléatoire
def generateRandomMomentum():
    speed = maxwell.rvs(scale=np.sqrt(k * T / mass))
    theta = 2 * np.pi * random()
    px = mass * speed * np.cos(theta)
    py = mass * speed * np.sin(theta)
    return vector(px, py, 0)

# Listes pour stocker les données
avg_momentum_list = []
single_electron_momentum = []
avg_direction_list = []  # Stocke la direction moyenne des électrons
single_electron_direction = []  # Stocke la direction d'un seul électron

# Boucle principale
Valeurs = 500

for iteration in range(Valeurs):
    rate(1000)

    # Déplace les sphères
    vitesse = []
    deltax = []
    for i in range(Natoms):
        vitesse.append(p[i] / mass)
        deltax.append(vitesse[i] * dt)
        Atoms[i].pos = apos[i] = apos[i] + deltax[i]

    for i in range(Natoms):
        loc = apos[i]
        if abs(loc.x) > L / 2:
            p[i].x *= -1
            apos[i].x = L / 2 * (1 if loc.x > 0 else -1)
        if abs(loc.y) > L / 2:
            p[i].y *= -1
            apos[i].y = L / 2 * (1 if loc.y > 0 else -1)

    # Identification des collisions avec les ions
    hitlist = checkIonCollisions()

    # Gestion des collisions
    for ij in hitlist:
        i = ij[0]
        j = ij[1]
        p[i] = generateRandomMomentum()
        apos[i] += (p[i] / mass) * dt

    # Calcul de la quantité de mouvement moyenne
    avg_momentum = np.mean([mag(p_i) for p_i in p])
    avg_momentum_list.append(avg_momentum)

    # Stocke la quantité de mouvement du premier électron
    single_electron_momentum.append(mag(p[0]))

    # Calcul de la direction moyenne des électrons
    avg_direction = np.mean([np.arctan2(p_i.y, p_i.x) for p_i in p])
    avg_direction_list.append(avg_direction)

    # Stocke la direction de l'électron choisi
    single_electron_direction.append(np.arctan2(p[0].y, p[0].x))

    # Recalcul de la température toutes les 10 itérations
    if iteration % 10 == 0:
        if avg_momentum > 1E-32:  # Si la quantité de mouvement moyenne est significativement non nulle
            # Calcul de la nouvelle température basée sur la quantité de mouvement
            T = (avg_momentum ** 2) * mass / (k * (DIM / 2))
        else:
            print("La quantité de mouvement est presque nulle, arrêt de la simulation.")
            break


def plot_momentum_decay(p0, tau, t_max, num_points=1000):
    t = np.linspace(0, t_max, num_points)  # Génère des points de temps
    p_t = p0 * np.exp(-t / tau)  # Calcule <p(t)>

    # Tracer la fonction de décroissance
    plt.plot(t, p_t, label='Décroissance Exponentielle', color='blue')

# Paramètres pour le tracé
p0 = avg_momentum_list[0] if avg_momentum_list else 1.0  # Valeur initiale de <p(t0)>
tau = 0.00000000001
t_max = Valeurs / 5000000
num_points = 100  # Nombre de points pour le tracé

start = 0  # Début
stop = Valeurs / 10  # Fin
step = 0.1  # Pas

# Calculer le nombre de points nécessaires
num_points = int((stop - start) / step)

# Utiliser linspace pour générer la liste
values = np.linspace(start, stop, num_points)

plt.figure() 
#sAppel de la fonction pour tracer la décroissance
#plot_momentum_decay(p0, tau, t_max, num_points)

# Tracé du graphique de la quantité de mouvement moyenne en fonction du temps
time_values = [dt * i for i in range(len(avg_momentum_list))]
plt.plot(time_values, avg_momentum_list)
plt.xlabel('Temps (s)')
plt.ylabel('Quantité de mouvement moyenne (kg.m/s)')
plt.title('Quantité de mouvement moyenne des électrons en fonction du temps')

plt.grid()
plt.show()

Voici on remarque que la quantité de mouvement moyenne descend exponentilement en fonction du temps et rhyme avec une baisse de température

### Dynamique sous l'effet d'une force externe ###

IX. _(10 points)_ &mdash; Pour passer de la cinétique à la dynamique des électrons libres, modifiez votre code de simulation en ajoutant une fonction qui applique un champ électrique uniforme. Celui-ci devra être de module ajustable et perpendiculaire à deux des côtés de la boîte. À chaque pas de temps $\mathrm{d}t$ sans collision, les électrons devront donc accélérer d'un incrément $\mathrm{d}p_x$ dicté par la force de Coulomb à leur position.

Copiez le code de votre fonction dans la cellule qui suit en n'oubliant pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

### Question statistique ###

X. _(5 points)_ &mdash; Pour quelques différents modules de champ électrique, présentez graphiquement l'évolution de la position moyenne des électrons en fonction du temps pour ses deux composantes parallèle et perpendiculaire au champ.

In [ ]:
import matplotlib.pyplot as plt

# Générer l'axe du temps
time = np.arange(max_iterations) * dt

# Tracer les positions moyennes parallèle et perpendiculaire au champ
plt.figure(figsize=(10, 5))

# Composante parallèle
plt.subplot(1, 2, 1)
plt.plot(time, positions_paralleles, label='Composante parallèle')
plt.xlabel('Temps (s)')
plt.ylabel('Position moyenne')
plt.title('Composante parallèle au champ électrique')
plt.legend()

# Ajouter une flèche pour le sens du champ
if E_direction == 'horizontal':
    if E_intensity > 0:
        arrow(pos=vector(0, 0.6, 0), axis=vector(0.5 * L, 0, 0), shaftwidth=0.05 * L, color=color.red)
    else:
        arrow(pos=vector(0, 0.6, 0), axis=vector(-0.5 * L, 0, 0), shaftwidth=0.05 * L, color=color.red)
    label = text(text='Champ horizontal', pos=vector(-0.25 * L, 0.68 * L, 0), height=0.05, color=color.red)
else:  # Cas pour un champ vertical
    if E_intensity > 0:
        arrow(pos=vector(-0.6, 0, 0), axis=vector(0, 0.5 * L, 0), shaftwidth=0.05 * L, color=color.red)
    else:
        arrow(pos=vector(-0.6, 0, 0), axis=vector(0, -0.5 * L, 0), shaftwidth=0.05 * L, color=color.red)
    label = text(text='Champ vertical', pos=vector(-0.68 * L, -0.20 * L, 0), height=0.05, color=color.red, axis=vector(0, 1, 0))

# Composante perpendiculaire
plt.subplot(1, 2, 2)
plt.plot(time, positions_perpendiculaires, label='Composante perpendiculaire')
plt.xlabel('Temps (s)')
plt.ylabel('Position moyenne')
plt.title('Composante perpendiculaire au champ électrique')
plt.legend()

# Affichage du graphique
plt.tight_layout()
plt.show()

# Médiagraphie #
- P. Drude, _Zur Elektronentheorie der Metalle; I Teil_, Annalen der Physik **306**(3), pp.566–613 (1900) https://doi.org/10.1002/andp.19003060312
- P. Drude, _Zur Elektronentheorie der Metalle; II Teil. Galvanomagnetische und Thermomagnetische Effecte_, Annalen der Physik **308**(11), pp.369–402 (1900) https://doi.org/10.1002/andp.19003081102
- P. Drude, _Zur Elektronentheorie der Metalle; Berichtigung_, Annalen der Physik **312**(3), pp.687–692 (1902) https://doi.org/10.1002/andp.19023120312
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies I_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.438-453 (1905) https://dwc.knaw.nl/DL/publications/PU00013989.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies II_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.585-593 (1905) https://dwc.knaw.nl/DL/publications/PU00014010.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies III_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.684-691 (1905) https://dwc.knaw.nl/DL/publications/PU00014024.pdf
 - La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le script fourni qui est exécuté dans ce _Notebook_ adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.
 
_**N.B. de débogage VPython:** Ayant plus codé en MATLAB qu'en Python jusqu'à maintenant, j'utilise Spyder qui offre une interface similaire, mais j'ai trouvé ce vidéo qui parle d'une <a href="https://www.youtube.com/watch?v=MJiVtz4Uj7M">installation VS code</a> qui peut-être aider? N'hésitez pas à partager de meilleures alternatives que j'ajouterai ici. Vous pouvez aussi tenter l'installation <a href="https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/">directement dans le Notebook</a>, mais au final chaque installation de distribution Python a son lot de <a href="https://xkcd.com/2347/">défis avec les dépendances</a>. Si rien ne fonctionne, n'hésitez pas à contacter la sympathique communauté qui développe et utilise cette librairie VPython d'abord via leur <a href="https://groups.google.com/g/vpython-users">groupe-forum</a>, puis leur <a href="https://github.com/vpython/vpython-jupyter">site GitHub</a> de dépôt de développement. J'ai toujours eu de bonnes expériences d'interaction avec eux!_ 
 